In [2]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

CODE = "if (x is not None) <mask> (x>1)"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
for output in outputs:
    print(output)

{'score': 0.7236990928649902, 'token': 8, 'token_str': ' and', 'sequence': 'if (x is not None) and(x>1)'}
{'score': 0.10633837431669235, 'token': 359, 'token_str': ' &', 'sequence': 'if (x is not None) &(x>1)'}
{'score': 0.021604055538773537, 'token': 463, 'token_str': 'and', 'sequence': 'if (x is not None)and(x>1)'}
{'score': 0.02122761495411396, 'token': 4248, 'token_str': ' AND', 'sequence': 'if (x is not None) AND(x>1)'}
{'score': 0.01699129305779934, 'token': 114, 'token_str': ' if', 'sequence': 'if (x is not None) if(x>1)'}
